In [2]:


import csv 
from ripser import ripser, lower_star_img
from persim import plot_diagrams
import numpy as np
from sklearn import datasets
import pandas as pd
import numpy as np
import random as ran
import scipy
from scipy import ndimage
import PIL
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


ListOfCharacter=[]

#Open csv file
with open('letters.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=' ')
    
    for row in reader:
        DataFromCSVLine = row[0].split(',')[1:]
        tenBytenMatrix = []
        for RowsInMatrix in range (10):
          tenBytenMatrix.append(DataFromCSVLine[RowsInMatrix*10:(RowsInMatrix+1)*10])
        
        ListOfCharacter.append(tenBytenMatrix)
        
#Below is the 10x10 matrix being matched to a variable matching the letter which the data represents
A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,Char1,Char2,Char3,Char4,Char5,Char6= (lett for lett in ListOfCharacter)
s=[A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,Char1,Char2,Char3,Char4,Char5,Char6]    
t=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','Period','Comma','Space','Dash','Colon','Semicolon']
j=0
ClassificationKey=[]
dataset=[]


#defines main function
def ClassifyALPH(i):
    index=[]
    j=s.index(i)    
    mat=np.array(i) #prodce single matrix per letter
    mat=mat.astype(np.float) #convert to float
    co = np.argwhere(mat!=0)# create matrix of coordinates 
    if len(co)==0: #if empty data set then all values are 0
        H1=0;R1=(0,0);R2=(0,0);RL0=0;RRL0=(0,0);RRL1=(0,0);RRL2=(0,0);LR0=0;RLR0=(0,0);RLR1=(0,0);RLR2=(0,0);TB0=0;RTB0=(0,0);RTB1=(0,0);RTB2=(0,0);
        BT0=0;RBT0=(0,0);RBT1=(0,0);RBT2=(0,0);LL0=0;LL1= 0;LL2=0;LLLS0=0;LLLS1=0;LLLS2=0;CO0=0;CO1= 0;CO2=0;COLS0=0;COLS1=0;COLS2=0;CI0=0;
        CI1= 0;CI2=0;CILS0=0;CILS1=0;CILS2=0;RLLS0= 0;RLLS1=0;RLLS2=0;LRLS0=0;LRLS1=0;LRLS2=0;TBLS0=0;TBLS1=0;TBLS2=0;BTLS0=0;BTLS1=0;BTLS2=0
        ranges= [RLLS0,RLLS1,RLLS2,LRLS0,LRLS1,LRLS2,TBLS0,TBLS1,TBLS2,BTLS0,BTLS1,BTLS2,LLLS0,LLLS1,LLLS2,COLS0,COLS1,COLS2,CILS0,CILS1,CILS2]  

    else:
        
        dgms = ripser(co)['dgms']#analyses the point cloud of coordinates in ripser 
        OneDset=(dgms[1:][0])#defines the range of 1D homology ""

        if len(OneDset)>0: #delete all repeated values
            OneDset=np.unique(OneDset, axis=0)          

        H1=len(OneDset)
        
        for h in np.arange(len(OneDset)):
            index1=(OneDset[h][1])
            index0=(OneDset[h][0])
            feat=np.subtract((OneDset[h][1]),(OneDset[h][0]))
            
            #removes noise in 1D
            if feat < 1:
                index.append(h)
                                    
        OneDset= np.delete(OneDset,index,0)
                      
        H1=(len(OneDset))          

        #RIGHT TO LEFT scan    
        RLmat=RightToLeft(mat) 
        dgm = lower_star_img(RLmat)
        RL0=len(dgm)
        RLLS0,RLLS1,RLLS2= USDictionaryUpdate(dgm)
        mat=np.array(i) 
        mat=mat.astype(np.float)
        #LEFT TO RIGHT
        LRmat=LeftToRight(mat) 
        dgm = lower_star_img(LRmat)
        LR0=len(dgm)
        LRLS0,LRLS1,LRLS2= USDictionaryUpdate(dgm)
        mat=np.array(i) 
        mat=mat.astype(np.float)
        
        #TOP TO BOTTOM
        TBmat=TopToBottom(mat)       
        dgm = lower_star_img(TBmat)
        TB0=len(dgm)
        TBLS0,TBLS1,TBLS2= USDictionaryUpdate(dgm)
        mat=np.array(i) 
        mat=mat.astype(np.float)
        
        #BOTTOM TO TOP
        BTmat=BottomToTop(mat) 
        dgm = lower_star_img(BTmat)
        BT0=len(dgm)
        BTLS0,BTLS1,BTLS2= USDictionaryUpdate(dgm)
        mat=np.array(i) 
        mat=mat.astype(np.float)
        
        #LOWER LEFT DIA
        LLmat= Diag_Array_LL(mat)
        dgm = lower_star_img(LLmat)
        LL0=len(dgm)
        LLLS0,LLLS1,LLLS2= USDictionaryUpdate(dgm)
        mat=np.array(i) 
        mat=mat.astype(np.float)
        
        #CENTER OUT
        COmat=CenterOutFiltration(mat)
        dgm = lower_star_img(COmat)
        CO0=len(dgm)
        COLS0,COLS1,COLS2= USDictionaryUpdate(dgm)
        mat=np.array(i) 
        mat=mat.astype(np.float)
        
        #CENTER IN
        CImat=CenterInFiltration(mat)
        dgm = lower_star_img(CImat)
        CI0=len(dgm)
        CILS0,CILS1,CILS2= USDictionaryUpdate(dgm)
        mat=np.array(i) 
        mat=mat.astype(np.float)
        
        ranges= [RLLS0,RLLS1,RLLS2,LRLS0,LRLS1,LRLS2,TBLS0,TBLS1,TBLS2,BTLS0,BTLS1,BTLS2,LLLS0,LLLS1,LLLS2,COLS0,COLS1,COLS2,CILS0,CILS1,CILS2]  
    #Populated Classification Key with all range values
    #Produces 3D vector (Letter#, Betti#, LifeSpan of Feature)
    #Each feature produces 1 3D vector
    for i in range(len(ranges)):
        if i<=2 and ranges[i]!=0.0:
            ClassificationKey.append([j,1,ranges[i]])
        elif 2<i<=5 and ranges[i]!=0.0:
            ClassificationKey.append([j,2,ranges[i]])
        elif 5<i<=8 and ranges[i]!=0.0:
            ClassificationKey.append([j,3,ranges[i]])
        elif 8<i<=11 and ranges[i]!=0.0:
            ClassificationKey.append([j,4,ranges[i]])
        elif 11<i<=14 and ranges[i]!=0.0:
            ClassificationKey.append([j,5,ranges[i]])
        elif 14<i<=17 and ranges[i]!=0.0:
            ClassificationKey.append([j,6,ranges[i]])
        elif 17<i<=20 and ranges[i]!=0.0:
            ClassificationKey.append([j,7,ranges[i]])

    #Polulate list with all lifespan values to later be input into pandas array
    ranges=list(ranges)
    dataset.append(ranges)    

    return ClassificationKey
    
    

def Diag_Array_LL(matrix): #lower left corner diagnal 
    AlteredMatrix =  np.zeros((10,10))
    
    BelowDiagonal = []
    AboveDiagonal =[]
    for diag_value in range(-9,10):
        if diag_value < 1:
            BelowDiagonal.append(np.diag(matrix,k=diag_value))
        else:
            AboveDiagonal.append(np.diag(matrix,k=diag_value))
            
    for array in BelowDiagonal:
        row = 0
        for column in range(10-len(array),10):
            if int(array[row]) == 0:
                AlteredMatrix[column,row] = 100
            else:
                AlteredMatrix[column,row] = int(array[row])*len(array)
            row +=1

    for array in AboveDiagonal:
        row = 0
        for column in range(10-len(array),10):
            if int(array[row]) == 0:
                AlteredMatrix[row,column] = 100
            else:
                AlteredMatrix[row,column] = int(array[row])*((10-len(array))+10)
            row +=1
  
    return AlteredMatrix

def CenterInFiltration(array):#Center in matrix creation
    AlteredMatrix =  np.zeros((10,10))
    startingSquare = 4
    endingSquare = 5
    mainHolder= []
    SquareValueHolder = []
    filteringArray1 = []
    filteringArray2 = []
                               
                               
    for numberofsquares in range(1,6):
        for x in range(startingSquare,endingSquare+1):
            for y in range(startingSquare,endingSquare+1):
                SquareValueHolder.append([x,y])
        
        startingSquare -=1
        endingSquare += 1

        if numberofsquares == 1:
            mainHolder.append(SquareValueHolder)

        elif numberofsquares == 2:
            mainHolder.append([i for i in SquareValueHolder if i not in mainHolder[0]])
            
        elif numberofsquares > 2:
            for arrays in mainHolder:
                SquareValueHolder = [i for i in SquareValueHolder if i not in arrays]
            mainHolder.append(SquareValueHolder)
            
        SquareValueHolder = []

    multiplier = 1 
    for element in reversed(mainHolder):
        for locations in reversed(element):
            if int(array[int(locations[0])][int(locations[1])]) * (multiplier) == 0:
                AlteredMatrix[int(locations[0])][int(locations[1])] = 100
            else:
                AlteredMatrix[int(locations[0])][int(locations[1])] = int(array[int(locations[0])][int(locations[1])]) * (multiplier)
        multiplier += 1 

    return (AlteredMatrix)

def CenterOutFiltration(array):#Center out matrix creation
    AlteredMatrix =  np.zeros((10,10))
    startingSquare = 4
    endingSquare = 5
    mainHolder= []
    SquareValueHolder = []
    filteringArray1 = []
    filteringArray2 = []
                               
                               
    for numberofsquares in range(1,6):
        for x in range(startingSquare,endingSquare+1):
            for y in range(startingSquare,endingSquare+1):
                SquareValueHolder.append([x,y])
        
        startingSquare -=1
        endingSquare += 1

        if numberofsquares == 1:
            mainHolder.append(SquareValueHolder)

        elif numberofsquares == 2:
            mainHolder.append([i for i in SquareValueHolder if i not in mainHolder[0]])
            
        elif numberofsquares > 2:
            for arrays in mainHolder:
                SquareValueHolder = [i for i in SquareValueHolder if i not in arrays]
            mainHolder.append(SquareValueHolder)
            
        SquareValueHolder = []

    multiplier = 1 
    for element in mainHolder:
        for locations in element:
            if int(array[int(locations[0])][int(locations[1])]) * (multiplier) == 0:
                AlteredMatrix[int(locations[0])][int(locations[1])] = 100
            else:
                AlteredMatrix[int(locations[0])][int(locations[1])] = int(array[int(locations[0])][int(locations[1])]) * (multiplier)
        multiplier += 1 

    return (AlteredMatrix)    
        

    
def LeftToRight(matrix):#Left to right martix creation
       
    for column in range(10):
        
        for row in range(10):
            if matrix[column][row] == 0.0:
                matrix[column][row]=100.0
               
            else:
                matrix[column][row]=float(row) +1.0
                
        
    return matrix
  
def RightToLeft(matrix):#Right to left matrix creation
       
    for row in range(10):        
        for column in range(10):
            if matrix[column][row] == 0.0:
                matrix[column][row]=100.0
               
            else:
                matrix[column][row]=10.0 -float(row) 
            
        
    return matrix

def BottomToTop(matrix): #Bottom top matrix creation
       
    for row in range(10):        
        for column in range(10):
            if matrix[column][row] == 0.0:
                matrix[column][row]=100.0
               
            else:
                matrix[column][row]=10.0 -float(column) 
           

    return matrix

def TopToBottom(matrix):#top bottom matrix creation
       
    for column in range(10):
        
        for row in range(10):
            if matrix[column][row] == 0.0:
                matrix[column][row]=100.0
               
            else:
                matrix[column][row]=float(column) +1.0
        
        
    return matrix

def USDictionaryUpdate(dgm):#find Lifespan from birth and death values, chance inf to 100.0
    if len(dgm) == 3:
        if dgm[0][1]==float('inf'):
            RA=(dgm[0][0]),(100.0)
        else:
            RA=(dgm[0][0]),(dgm[0][1])
        
        if dgm[1][1]==float('inf'):
            RB=(dgm[1][0]),(100.0)
        else:
            RB=(dgm[1][0]),(dgm[1][1])
            
        if dgm[2][1]==float('inf'):
            RC=(dgm[2][0]),(100.0)
        else:
            RC=(dgm[2][0]),(dgm[2][1])
        
        
    elif len(dgm) == 2:
        if dgm[0][1]==float('inf'):
            RA=(dgm[0][0]),(100.0)
        else:
            RA=(dgm[0][0]),(dgm[0][1])
        
        if dgm[1][1]==float('inf'):
            RB=(dgm[1][0]),(100.0)
        else:
            RB=(dgm[1][0]),(dgm[1][1])
        RC=(0,0)
    elif len(dgm) == 1:
        if dgm[0][1]==float('inf'):
            RA=(dgm[0][0]),(100.0)
        else:
            RA=(dgm[0][0]),(dgm[0][1])
        
        RB=(0,0)
        RC=(0,0)
    else:
        RA=(0,0)
        RB=(0,0)
        RC=(0,0)
    LSA= RA[1]-RA[0]
    LSB= RB[1]-RB[0]
    LSC= RC[1]-RC[0]
    return(LSA,LSB,LSC)
    
#run main function
dictionary=[ClassifyALPH(i) for i in s]
ClassificationKey=np.array(ClassificationKey)
ClassificationOutput=pd.DataFrame(dataset,columns=["RLLS0","RLLS1","RLLS2", "LRLS0","LRLS1","LRLS2","TBLS0",'TBLS1','TBLS2','BTLS0','BTLS1','BTLS2','LLLS0','LLLS1','LLLS2','COLS0','COLS1','COLS2','CILS0','CILS1','CILS2'])
DuplicatedFeatureMat=ClassificationOutput[ClassificationOutput.duplicated()]


ClassificationOutput


C:\Anaconda3\lib\site-packages\ripser\ripser.py:220: UserWarning: The input point cloud has more columns than rows; did you mean to transpose?
  "The input point cloud has more columns than rows; "
C:\Anaconda3\lib\site-packages\ripser\ripser.py:216: UserWarning: The input matrix is square, but the distance_matrix flag is off.  Did you mean to indicate that this was a distance matrix?
  + "this was a distance matrix?"


,RLLS0,RLLS1,RLLS2,LRLS0,LRLS1,LRLS2,TBLS0,TBLS1,TBLS2,BTLS0,...,BTLS2,LLLS0,LLLS1,LLLS2,COLS0,COLS1,COLS2,CILS0,CILS1,CILS2
0,97.0,0.0,0.0,98.0,0.0,0,99.0,0.0,0.0,3.0,...,0.0,1.0,97.0,0.0,99.0,0.0,0.0,2.0,2.0,99.0
1,1.0,97.0,0.0,97.0,0.0,0,99.0,0.0,0.0,98.0,...,0.0,96.0,0.0,0.0,99.0,0.0,0.0,1.0,99.0,0.0
2,5.0,97.0,0.0,98.0,0.0,0,98.0,0.0,0.0,98.0,...,0.0,95.0,0.0,0.0,98.0,0.0,0.0,1.0,1.0,98.0
3,98.0,0.0,0.0,97.0,0.0,0,98.0,0.0,0.0,98.0,...,0.0,96.0,0.0,0.0,1.0,97.0,0.0,1.0,1.0,98.0
4,5.0,5.0,97.0,97.0,0.0,0,98.0,0.0,0.0,98.0,...,0.0,96.0,0.0,0.0,99.0,0.0,0.0,1.0,2.0,98.0
5,4.0,96.0,0.0,97.0,0.0,0,98.0,0.0,0.0,98.0,...,0.0,96.0,0.0,0.0,99.0,0.0,0.0,1.0,1.0,98.0
6,4.0,96.0,0.0,98.0,0.0,0,98.0,0.0,0.0,97.0,...,0.0,95.0,0.0,0.0,99.0,0.0,0.0,1.0,98.0,0.0
7,96.0,0.0,0.0,97.0,0.0,0,2.0,98.0,0.0,4.0,...,0.0,3.0,95.0,0.0,98.0,0.0,0.0,1.0,2.0,98.0
8,1.0,95.0,0.0,1.0,96.0,0,98.0,0.0,0.0,98.0,...,0.0,95.0,0.0,0.0,99.0,0.0,0.0,3.0,98.0,0.0
9,95.0,0.0,0.0,2.0,96.0,0,98.0,0.0,0.0,97.0,...,0.0,1.0,94.0,0.0,99.0,0.0,0.0,2.0,98.0,0.0


In [3]:
DuplicatedFeatureMat

,RLLS0,RLLS1,RLLS2,LRLS0,LRLS1,LRLS2,TBLS0,TBLS1,TBLS2,BTLS0,...,BTLS2,LLLS0,LLLS1,LLLS2,COLS0,COLS1,COLS2,CILS0,CILS1,CILS2
